# Publish & Subscribe with MQTT 

In this notebook we will use PAHO as our MQTT broker.

- https://pypi.org/project/paho-mqtt/


## Contents
1. Steve's internet guide tutorial
2. Some more basic scripts

## 0. Install packages

In [1]:
!pip install paho-mqtt

     |████████████████████████████████| 99 kB 4.2 MB/s eta 0:00:01
  Created wheel for paho-mqtt: filename=paho_mqtt-1.6.1-py3-none-any.whl size=62133 sha256=acd3a31d1ffc62a3666d68fe66145b4266b8f0b50e1b6fc4b0fbf483852bb0e2
  Stored in directory: /Users/michielbontenbal/Library/Caches/pip/wheels/0f/90/29/db29bb8ddc98ec5f2363b959130c9ddbcf5cfdb4a00b6184dd
Successfully built paho-mqtt


## 1. Steve's internet guide tutorial

In [2]:
# source: http://www.steves-internet-guide.com/python-mqtt-publish-subscribe/
# source: http://www.steves-internet-guide.com/mqtt-username-password-example/ => if you want to use eclipse you need account+pw

import time
import paho.mqtt.client as paho
#broker="broker.hivemq.com"
broker="mqtt.eclipse.org"
#define callback
def on_message(client, userdata, message):
    time.sleep(1)
    print("received message =",str(message.payload.decode("utf-8")))

client= paho.Client("client-001") #create client object client1.on_publish = on_publish #assign function to callback client1.connect(broker,port) #establish connection client1.publish("house/bulb1","on")
######Bind function to callback
client.on_message=on_message
#####
print("connecting to broker ",broker)
client.connect(broker)#connect
client.loop_start() #start loop to process received messages
print("subscribing ")
client.subscribe("house/bulb1")#subscribe
time.sleep(2)
print("publishing ")
client.publish("house/bulb1","on")#publish
time.sleep(4)
client.disconnect() #disconnect
client.loop_stop() #stop loop


connecting to broker  mqtt.eclipse.org


TimeoutError: [WinError 10060] A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond

In [9]:
#source: http://www.steves-internet-guide.com/into-mqtt-python-client/
import paho.mqtt.client as mqtt #import the client1
broker_address="mqtt.eclipseprojects.io" #use external broker
client = mqtt.Client("P1") #create new instance
client.connect(broker_address) #connect to broker
client.publish("house/main-light","OFF")#publish
print('done')

done


In [10]:
#source: http://www.steves-internet-guide.com/into-mqtt-python-client/
import paho.mqtt.client as mqtt #import the client1
broker_address="mqtt.eclipseprojects.io" #use external broker
print("creating new instance")
client = mqtt.Client("P1") #create new instance
print("connecting to broker")
client.connect(broker_address) #connect to broker
print("Subscribing to topic","house/bulbs/bulb1")
client.subscribe("house/bulbs/bulb1")
print("Publishing message to topic","house/bulbs/bulb1")
client.publish("house/bulbs/bulb1","OFF")

creating new instance
connecting to broker
Subscribing to topic house/bulbs/bulb1
Publishing message to topic house/bulbs/bulb1


## 2. Some basic scripts

Source: Dutch Book - IoT with the Raspberry Pi

In [2]:
#basis script publisher zie p.250
import paho.mqtt.client as mqtt

mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

client = mqtt.Client()

client.connect(mqtt_server_ip, 1883) #use port 1883 for unsafe 
client.publish(mqtt_topic, '20 graden')
client.disconnect
print('done')

done


In [ ]:
#basis script subscriber => create a separate python file (or notebook) and run it. Then run the previous cell.
import paho.mqtt.client as mqtt
mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

def on_connect(client, userdata, flags, rc):
    print('verbonden')
    client.subscribe(mqtt_topic)
def on_message(client, userdata, msg):
    print(msg.topic +" "+ str(msg.payload))
client = mqtt.Client()
client.on_connect = on_connect
client.on_message = on_message

client.connect(mqtt_server_ip, 1883, 60)
client.loop_forever()
client.disconnect

verbonden


### 2b. Varianten op dit script

In [2]:
# as above but now with a while loop
import psutil
from time import sleep
import paho.mqtt.client as mqtt

mqtt_topic = 'roomTemp'
mqtt_server_ip="mqtt.eclipseprojects.io" #use external broker

client = mqtt.Client()

for i in range(0,3): # for loop that will run three times.
    cpu_use = psutil.cpu_percent()
    client.connect(mqtt_server_ip, 1883)
    #client.publish(mqtt_topic, "cpu use =")
    client.publish(mqtt_topic, cpu_use)
    client.disconnect
    print(cpu_use)
    sleep(5)
    
print('done')

0.0
11.4
9.9
done


In [4]:
# as in first script but now with json message string
# This is just to show the script copy it to a new file to properly run it
import time
import paho.mqtt.client as mqtt
import json

mqtt_topic = 'roomTemp'
mqtt_server_ip = 'mqtt.eclipseprojects.io'

client = mqtt.Client()

msg_json = {
    "app_id": "test_project", "dev_id": "test_device",
    "payload_fields": {"temperature": 42, "co2": 42},
    "time": int(time.time() * 1e3)
}
msg_str = json.dumps(msg_json)
print(msg_str)
client.connect(mqtt_server_ip, 1883)
client.publish(mqtt_topic, msg_str)
client.disconnect()
print('done')

{"app_id": "test_project", "dev_id": "test_device", "payload_fields": {"temperature": 42, "co2": 42}, "time": 1629271915767}
done


In [5]:
# as in first script but now with json message string AND Try Except
# as in first script but now with json message string
# This is just to show the script copy it to a new file to properly run it
import time
import paho.mqtt.client as mqtt
import json

mqtt_topic = 'roomTemp'
mqtt_server_ip = 'mqtt.eclipseprojects.io'

client = mqtt.Client()

msg_json = {
    "app_id": "test_project", "dev_id": "test_device",
    "payload_fields": {"temperature": 42, "co2": 42},
    "time": int(time.time() * 1e3)
}
msg_str = json.dumps(msg_json)
print(msg_str)
try:
    client.connect(mqtt_server_ip, 1883)
    client.publish(mqtt_topic, msg_str)
    print('message sent')
    client.disconnect()
except:
    pass
print('done')

{"app_id": "test_project", "dev_id": "test_device", "payload_fields": {"temperature": 42, "co2": 42}, "time": 1629271996401}
message sent
done
